# 合并掩码文件

In [43]:
import os
import re
import cv2
import numpy as np
import shutil
from collections import defaultdict
from pathlib import Path

def merge_masks():
    # 要处理的目录
    masks_dir = Path("D:/Dataset/New_CBDDSM/Dmass/test/masks")
    
    # 分别处理良性和恶性文件夹
    for class_dir in ["benign", "malignant"]:
        class_path = masks_dir / class_dir
        
        if not class_path.exists():
            print(f"目录 {class_path} 不存在")
            continue
        
        # 获取所有掩码文件
        mask_files = [f for f in class_path.glob("*.png")]
        print(f"在 {class_dir} 类别中找到 {len(mask_files)} 个掩码文件")
        
        # 使用正则表达式解析文件名模式
        # 修改正则表达式匹配像 "Mass-Training_P_00044_RIGHT_CC" 这样的基本名称
        pattern = re.compile(r"(Mass-Test_P_\d+_[A-Z]+_[A-Z]+)_\d+(_\d+-\d+\.png)")
        
        # 将文件按基本名称分组 (不包括序列号)
        grouped_masks = defaultdict(list)
        
        for mask_file in mask_files:
            match = pattern.match(mask_file.name)
            if match:
                # 只提取基本名称部分，忽略序列号
                base_name = match.group(1)
                grouped_masks[base_name].append(mask_file)
            else:
                # 对不符合模式的文件不处理
                print(f"文件 {mask_file.name} 不符合命名模式，将保持不变")
        
        # 处理每组掩码
        merged_count = 0
        deleted_count = 0
        
        for base_name, files in grouped_masks.items():
            if len(files) > 1:
                # 需要合并的情况
                merged_mask = None
                
                for file_path in files:
                    mask = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
                    
                    if merged_mask is None:
                        merged_mask = np.zeros_like(mask)
                    
                    # 合并掩码（取最大值）
                    merged_mask = np.maximum(merged_mask, mask)
                
                # 保存合并后的掩码到一个临时文件
                temp_file = class_path / f"{base_name}_temp.png"
                cv2.imwrite(str(temp_file), merged_mask)
                
                # 删除原始文件
                for file_path in files:
                    os.remove(file_path)
                    deleted_count += 1
                
                # 重命名临时文件为最终文件名
                final_file = class_path / f"{base_name}.png"
                os.rename(temp_file, final_file)
                
                merged_count += 1
                print(f"已合并 {len(files)} 个掩码文件到 {final_file.name}")
        
        print(f"已在 {class_dir} 类别中合并 {merged_count} 组文件，删除了 {deleted_count} 个原始文件")

if __name__ == "__main__":
    merge_masks()
    print("掩码合并完成！原始掩码文件已删除！")

在 benign 类别中找到 231 个掩码文件
已合并 2 个掩码文件到 Mass-Test_P_00173_RIGHT_CC.png
已合并 2 个掩码文件到 Mass-Test_P_00173_RIGHT_MLO.png
已合并 2 个掩码文件到 Mass-Test_P_00343_RIGHT_CC.png
已合并 2 个掩码文件到 Mass-Test_P_00343_RIGHT_MLO.png
已合并 2 个掩码文件到 Mass-Test_P_00928_RIGHT_MLO.png
已合并 3 个掩码文件到 Mass-Test_P_01090_LEFT_CC.png
已合并 4 个掩码文件到 Mass-Test_P_01090_LEFT_MLO.png
已合并 2 个掩码文件到 Mass-Test_P_01566_RIGHT_CC.png
已合并 3 个掩码文件到 Mass-Test_P_01566_RIGHT_MLO.png
已在 benign 类别中合并 9 组文件，删除了 22 个原始文件
在 malignant 类别中找到 147 个掩码文件
已合并 2 个掩码文件到 Mass-Test_P_00116_RIGHT_CC.png
已合并 2 个掩码文件到 Mass-Test_P_00116_RIGHT_MLO.png
已在 malignant 类别中合并 2 组文件，删除了 4 个原始文件
掩码合并完成！原始掩码文件已删除！


# 重命名掩码文件

In [44]:
import os
import re
import shutil
from pathlib import Path

def rename_masks():
    # 要处理的目录
    masks_dir = Path("D:/Dataset/New_CBDDSM/Dmass/test/masks")
    
    # 分别处理良性和恶性文件夹
    for class_dir in ["benign", "malignant"]:
        class_path = masks_dir / class_dir
        
        if not class_path.exists():
            print(f"目录 {class_path} 不存在")
            continue
        
        # 获取所有掩码文件
        mask_files = [f for f in class_path.glob("*.png")]
        print(f"在 {class_dir} 类别中找到 {len(mask_files)} 个掩码文件")
        
        # 使用正则表达式解析文件名模式
        # 修改正则表达式匹配像 "Mass-Training_P_00044_RIGHT_CC_1_1-2.png" 这样的文件名
        pattern = re.compile(r"(Mass-Test_P_\d+_[A-Z]+_[A-Z]+)_\d+(_\d+-\d+\.png)")
        
        # 重命名计数
        renamed_count = 0
        skipped_count = 0
        
        # 临时目录，用于避免重名问题
        temp_dir = class_path / "temp_rename"
        temp_dir.mkdir(exist_ok=True)
        
        # 第一步：重命名到临时文件
        for mask_file in mask_files:
            match = pattern.match(mask_file.name)
            if match:
                # 提取基本名称部分
                base_name = match.group(1)
                new_name = f"{base_name}.png"
                
                # 先移动到临时目录
                temp_path = temp_dir / new_name
                shutil.copy2(mask_file, temp_path)
                os.remove(mask_file)
                renamed_count += 1
            else:
                # 对不符合模式的文件不处理
                print(f"文件 {mask_file.name} 不符合命名模式，将保持不变")
                skipped_count += 1
        
        # 第二步：从临时目录移回原目录
        for temp_file in temp_dir.glob("*.png"):
            target_file = class_path / temp_file.name
            if target_file.exists():
                print(f"警告：文件 {target_file} 已存在，将被覆盖")
            shutil.move(temp_file, target_file)
        
        # 删除临时目录
        temp_dir.rmdir()
        
        print(f"在 {class_dir} 类别中重命名了 {renamed_count} 个文件，跳过了 {skipped_count} 个文件")

if __name__ == "__main__":
    rename_masks()
    print("掩码重命名完成！")

在 benign 类别中找到 218 个掩码文件
文件 Mass-Test_P_00173_RIGHT_CC.png 不符合命名模式，将保持不变
文件 Mass-Test_P_00173_RIGHT_MLO.png 不符合命名模式，将保持不变
文件 Mass-Test_P_00343_RIGHT_CC.png 不符合命名模式，将保持不变
文件 Mass-Test_P_00343_RIGHT_MLO.png 不符合命名模式，将保持不变
文件 Mass-Test_P_00928_RIGHT_MLO.png 不符合命名模式，将保持不变
文件 Mass-Test_P_01090_LEFT_CC.png 不符合命名模式，将保持不变
文件 Mass-Test_P_01090_LEFT_MLO.png 不符合命名模式，将保持不变
文件 Mass-Test_P_01566_RIGHT_CC.png 不符合命名模式，将保持不变
文件 Mass-Test_P_01566_RIGHT_MLO.png 不符合命名模式，将保持不变
在 benign 类别中重命名了 209 个文件，跳过了 9 个文件
在 malignant 类别中找到 145 个掩码文件
文件 Mass-Test_P_00116_RIGHT_CC.png 不符合命名模式，将保持不变
文件 Mass-Test_P_00116_RIGHT_MLO.png 不符合命名模式，将保持不变
在 malignant 类别中重命名了 143 个文件，跳过了 2 个文件
掩码重命名完成！


# 重命名原始图像文件

In [45]:
import os
import re
import shutil
from pathlib import Path

def rename_images():
    # 要处理的目录
    images_dir = Path("D:/Dataset/New_CBDDSM/Dmass/test/images")
    
    # 分别处理良性和恶性文件夹
    for class_dir in ["benign", "malignant"]:
        class_path = images_dir / class_dir
        
        if not class_path.exists():
            print(f"目录 {class_path} 不存在")
            continue
        
        # 获取所有图像文件
        image_files = [f for f in class_path.glob("*.png")]
        print(f"在 {class_dir} 类别中找到 {len(image_files)} 个图像文件")
        
        # 更新正则表达式匹配像 "Mass-Training_P_00004_LEFT_CC_1-1.png" 这样的文件名
        pattern = re.compile(r"(Mass-Test_P_\d+_[A-Z]+_[A-Z]+)_(\d+-\d+\.png)")
        
        # 重命名计数
        renamed_count = 0
        skipped_count = 0
        
        # 临时目录，用于避免重名问题
        temp_dir = class_path / "temp_rename"
        temp_dir.mkdir(exist_ok=True)
        
        # 第一步：重命名到临时文件
        for image_file in image_files:
            match = pattern.match(image_file.name)
            if match:
                # 提取基本名称部分
                base_name = match.group(1)
                new_name = f"{base_name}.png"
                
                # 先移动到临时目录
                temp_path = temp_dir / new_name
                shutil.copy2(image_file, temp_path)
                os.remove(image_file)
                renamed_count += 1
            else:
                # 对不符合模式的文件不处理
                print(f"文件 {image_file.name} 不符合命名模式，将保持不变")
                skipped_count += 1
        
        # 第二步：从临时目录移回原目录
        for temp_file in temp_dir.glob("*.png"):
            target_file = class_path / temp_file.name
            if target_file.exists():
                print(f"警告：文件 {target_file} 已存在，将被覆盖")
            shutil.move(temp_file, target_file)
        
        # 删除临时目录
        temp_dir.rmdir()
        
        print(f"在 {class_dir} 类别中重命名了 {renamed_count} 个文件，跳过了 {skipped_count} 个文件")

if __name__ == "__main__":
    rename_images()
    print("原始图像重命名完成！")

在 benign 类别中找到 218 个图像文件
在 benign 类别中重命名了 218 个文件，跳过了 0 个文件
在 malignant 类别中找到 143 个图像文件
在 malignant 类别中重命名了 143 个文件，跳过了 0 个文件
原始图像重命名完成！


# 原图/掩码数量匹配

In [46]:
import os
from pathlib import Path
import pandas as pd

def check_image_mask_matching():
    """检查图像文件和掩码文件是否一一对应"""
    
    # 设置目录路径
    base_dir = Path("D:/Dataset/New_CBDDSM/Dmass/test")
    images_dir = base_dir / "images"
    masks_dir = base_dir / "masks"
    
    # 存储不匹配的文件
    missing_masks = []
    missing_images = []
    
    # 分别检查良性和恶性文件夹
    for class_dir in ["benign", "malignant"]:
        print(f"\n检查 {class_dir} 类别文件的匹配情况...")
        
        # 设置当前类别的目录
        img_class_dir = images_dir / class_dir
        mask_class_dir = masks_dir / class_dir
        
        if not img_class_dir.exists():
            print(f"警告: 图像目录 {img_class_dir} 不存在!")
            continue
            
        if not mask_class_dir.exists():
            print(f"警告: 掩码目录 {mask_class_dir} 不存在!")
            continue
            
        # 获取图像和掩码文件名列表
        image_files = set([f.stem for f in img_class_dir.glob("*.png")])
        mask_files = set([f.stem for f in mask_class_dir.glob("*.png")])
        
        print(f"图像文件数量: {len(image_files)}")
        print(f"掩码文件数量: {len(mask_files)}")
        
        # 查找缺失的掩码文件
        for img_file in image_files:
            if img_file not in mask_files:
                missing_file = f"{class_dir}/{img_file}.png"
                missing_masks.append(missing_file)
        
        # 查找缺失的图像文件
        for mask_file in mask_files:
            if mask_file not in image_files:
                missing_file = f"{class_dir}/{mask_file}.png"
                missing_images.append(missing_file)
        
        # 输出当前类别的匹配结果
        if not missing_masks and not missing_images:
            print(f"{class_dir} 类别下的所有文件完全匹配！")
        else:
            if missing_masks:
                print(f"{class_dir} 类别下有 {len([m for m in missing_masks if m.startswith(class_dir)])} 个图像没有对应的掩码。")
            if missing_images:
                print(f"{class_dir} 类别下有 {len([m for m in missing_images if m.startswith(class_dir)])} 个掩码没有对应的图像。")
    
    # 显示详细的不匹配文件列表
    print("\n" + "="*50)
    print("详细的不匹配文件列表:")
    print("="*50)
    
    if missing_masks:
        print("\n以下图像文件缺少对应的掩码文件:")
        for idx, file in enumerate(missing_masks, 1):
            print(f"{idx}. images/{file}")
    
    if missing_images:
        print("\n以下掩码文件缺少对应的图像文件:")
        for idx, file in enumerate(missing_images, 1):
            print(f"{idx}. masks/{file}")
    
    # 汇总结果
    print("\n" + "="*50)
    print("匹配结果汇总:")
    print(f"总计 {len(missing_masks)} 个图像文件缺少对应的掩码")
    print(f"总计 {len(missing_images)} 个掩码文件缺少对应的图像")
    
    # # 将不匹配的文件列表保存为CSV文件（可选）
    # if missing_masks or missing_images:
    #     # 创建数据框保存结果
    #     if missing_masks:
    #         df_missing_masks = pd.DataFrame({
    #             "文件类型": ["图像"] * len(missing_masks),
    #             "文件路径": [f"images/{f}" for f in missing_masks]
    #         })
    #     else:
    #         df_missing_masks = pd.DataFrame(columns=["文件类型", "文件路径"])
            
    #     if missing_images:
    #         df_missing_images = pd.DataFrame({
    #             "文件类型": ["掩码"] * len(missing_images),
    #             "文件路径": [f"masks/{f}" for f in missing_images]
    #         })
    #     else:
    #         df_missing_images = pd.DataFrame(columns=["文件类型", "文件路径"])
        
    #     # 合并数据框
    #     df_all = pd.concat([df_missing_masks, df_missing_images], ignore_index=True)
        
    #     # 保存为CSV文件
    #     output_file = base_dir / "unmatched_files.csv"
    #     df_all.to_csv(output_file, index=False, encoding="utf-8-sig")
    #     print(f"\n不匹配的文件列表已保存到: {output_file}")
    
    return len(missing_masks) == 0 and len(missing_images) == 0







if __name__ == "__main__":
    all_matched = check_image_mask_matching()
    if all_matched:
        print("\n所有图像和掩码文件都完全匹配！")
    else:
        print("\n存在不匹配的文件，请检查上述列表处理这些不匹配的文件。")


检查 benign 类别文件的匹配情况...
图像文件数量: 218
掩码文件数量: 218
benign 类别下的所有文件完全匹配！

检查 malignant 类别文件的匹配情况...
图像文件数量: 143
掩码文件数量: 145
malignant 类别下有 2 个掩码没有对应的图像。

详细的不匹配文件列表:

以下掩码文件缺少对应的图像文件:
1. masks/malignant/Mass-Test_P_00969_LEFT_MLO.png
2. masks/malignant/Mass-Test_P_00969_LEFT_CC.png

匹配结果汇总:
总计 0 个图像文件缺少对应的掩码
总计 2 个掩码文件缺少对应的图像

存在不匹配的文件，请检查上述列表处理这些不匹配的文件。


In [47]:
import os
from pathlib import Path

def delete_orphaned_masks():
    """删除没有对应图像文件的掩码文件"""
    
    # 设置目录路径
    base_dir = Path("D:/Dataset/New_CBDDSM/Dmass/test")
    images_dir = base_dir / "images"
    masks_dir = base_dir / "masks"
    
    # 存储要删除的掩码文件
    masks_to_delete = []
    
    # 分别检查良性和恶性文件夹
    for class_dir in ["benign", "malignant"]:
        print(f"\n检查 {class_dir} 类别文件...")
        
        # 设置当前类别的目录
        img_class_dir = images_dir / class_dir
        mask_class_dir = masks_dir / class_dir
        
        if not img_class_dir.exists():
            print(f"警告: 图像目录 {img_class_dir} 不存在!")
            continue
            
        if not mask_class_dir.exists():
            print(f"警告: 掩码目录 {mask_class_dir} 不存在!")
            continue
            
        # 获取图像和掩码文件名列表
        image_files = set([f.stem for f in img_class_dir.glob("*.png")])
        mask_files = [f for f in mask_class_dir.glob("*.png")]
        
        print(f"图像文件数量: {len(image_files)}")
        print(f"掩码文件数量: {len(mask_files)}")
        
        # 查找缺少对应图像的掩码文件
        orphaned_masks = []
        for mask_file in mask_files:
            if mask_file.stem not in image_files:
                orphaned_masks.append(mask_file)
                masks_to_delete.append(mask_file)
        
        if orphaned_masks:
            print(f"在 {class_dir} 类别中找到 {len(orphaned_masks)} 个没有对应图像的掩码文件")
        else:
            print(f"在 {class_dir} 类别中所有掩码文件都有对应的图像文件")
    
    # 显示要删除的文件
    if masks_to_delete:
        print("\n" + "="*50)
        print(f"将删除以下 {len(masks_to_delete)} 个无对应图像的掩码文件:")
        for idx, mask_file in enumerate(masks_to_delete, 1):
            print(f"{idx}. {mask_file}")
        
        # 确认是否删除
        confirm = input("\n确认删除这些文件? (y/n): ").strip().lower()
        if confirm == 'y':
            # 执行删除
            for mask_file in masks_to_delete:
                os.remove(mask_file)
                print(f"已删除: {mask_file}")
            print(f"\n成功删除了 {len(masks_to_delete)} 个无对应图像的掩码文件")
        else:
            print("\n操作已取消，未删除任何文件")
    else:
        print("\n没有找到需要删除的掩码文件，所有掩码文件都有对应的图像文件")

if __name__ == "__main__":
    delete_orphaned_masks()


检查 benign 类别文件...
图像文件数量: 218
掩码文件数量: 218
在 benign 类别中所有掩码文件都有对应的图像文件

检查 malignant 类别文件...
图像文件数量: 143
掩码文件数量: 145
在 malignant 类别中找到 2 个没有对应图像的掩码文件

将删除以下 2 个无对应图像的掩码文件:
1. D:\Dataset\New_CBDDSM\Dmass\test\masks\malignant\Mass-Test_P_00969_LEFT_CC.png
2. D:\Dataset\New_CBDDSM\Dmass\test\masks\malignant\Mass-Test_P_00969_LEFT_MLO.png
已删除: D:\Dataset\New_CBDDSM\Dmass\test\masks\malignant\Mass-Test_P_00969_LEFT_CC.png
已删除: D:\Dataset\New_CBDDSM\Dmass\test\masks\malignant\Mass-Test_P_00969_LEFT_MLO.png

成功删除了 2 个无对应图像的掩码文件
